In [59]:
# pip install xgboost
# pip install hyperopt
#https://hyperopt.github.io/hyperopt/?source=post_page
# pip install category_encoders
# pip install scikit-optimize
# pip install dtreeviz


**Install Libraries**

In [60]:
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb


import shap

from xgboost import plot_importance
from xgboost import XGBRegressor, plot_importance, plot_tree, plotting


import dtreeviz
import graphviz



from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
import scipy.stats as stats
from sklearn.pipeline import Pipeline
from category_encoders.target_encoder import TargetEncoder



# #for hyperparameter tuning
# import hyperopt
# from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

from sklearn.preprocessing import StandardScaler




**Load datasets**

In [61]:
def load_dataset(data):
    df = pd.read_csv(data)
   
    return df


In [62]:
df_bb= load_dataset('cleaned_files/basketball_clean_supervised.csv')
df_fb= load_dataset('cleaned_files/football_clean_supervised.csv')

In [63]:
df_fb.head(3)

,Unnamed: 0,NAME,GRADE,AGE,SKILL,STARCOLL,NUMOFF,POS,HEIGHT_IN,WEIGHT_LBS,...,TIK_LONG,TOT_FOL,SPORT,RECRUIT_YEAR,EXP_MONTHS,EXP_YEARS,institution_name_short,ClassificationCode,REV_MEN,EXP_MEN
0,0,Arch Manning,2.0,18.28281,99.53,Texas,18.0,1.0,75.5,220.0,...,8900.0,73900.0,football,2023.0,2.0,0.189041,Texas,1.0,161532860.0,50633156.0
1,1,Anthony Hill,2.0,18.00000,97.87,Texas,25.0,8.0,73.5,225.0,...,0.0,13100.0,football,2023.0,2.0,0.189041,Texas,1.0,161532860.0,50633156.0
2,2,CJ Baxter Jr.,2.0,18.00000,97.26,Texas,25.0,9.0,73.0,216.0,...,1700.0,10900.0,football,2023.0,2.0,0.189041,Texas,1.0,161532860.0,50633156.0


In [64]:
#save athlete names for later indexing
athletes_fb = df_fb['NAME']
athletes_bb = df_bb['NAME']
# athlete_names

df_bb = df_bb.drop(columns=['Unnamed: 0', 'NAME','SPORT','STARCOLL','EXP_YEARS','institution_name_short'])
df_fb= df_fb.drop(columns=['Unnamed: 0', 'NAME','SPORT', 'STARCOLL','EXP_YEARS','institution_name_short'])


In [65]:

df_fb.columns
# df_fb.info()

Index(['GRADE', 'AGE', 'SKILL', 'NUMOFF', 'POS', 'HEIGHT_IN', 'WEIGHT_LBS',
       'COLLDIST_MI', 'NILVAL_LONG_USD', 'INSTA_LONG', 'TWIT_LONG', 'TIK_LONG',
       'TOT_FOL', 'RECRUIT_YEAR', 'EXP_MONTHS', 'ClassificationCode',
       'REV_MEN', 'EXP_MEN'],
      dtype='object')

In [66]:
# Extract target and features
Target = 'NILVAL_LONG_USD'
Predictors = ['GRADE', 'AGE', 'SKILL', 'NUMOFF', 'POS', 'HEIGHT_IN', 'WEIGHT_LBS',
       'COLLDIST_MI', 'INSTA_LONG', 'TWIT_LONG', 'TIK_LONG',
       'TOT_FOL', 'RECRUIT_YEAR', 'EXP_MONTHS', 'EXP_YEARS',
       'ClassificationCode', 'REV_MEN', 'EXP_MEN']

X = df_fb.drop('NILVAL_LONG_USD', axis=1).values
y =  df_fb[['NILVAL_LONG_USD']].values
# X.head(3)

In [67]:
#split the data
#Shuffle data given concern that dataset has athletes ordered by SKILL (but also look at skew later based on model performance)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True)
print(X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(1010, 17) (1010, 1)
(253, 17) (253, 1)


In [68]:
# Extract target and features
Target = 'NILVAL_LONG_USD'
Predictors = ['GRADE', 'AGE', 'SKILL', 'NUMOFF', 'POS', 'HEIGHT_IN', 'WEIGHT_LBS',
       'COLLDIST_MI', 'INSTA_LONG', 'TWIT_LONG', 'TIK_LONG',
       'TOT_FOL', 'RECRUIT_YEAR', 'EXP_MONTHS', 'EXP_YEARS',
       'ClassificationCode', 'REV_MEN', 'EXP_MEN']

X = df_fb.drop('NILVAL_LONG_USD', axis=1).values
y =  df_fb[['NILVAL_LONG_USD']].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True)
print(X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(1010, 17) (1010, 1)
(253, 17) (253, 1)


**Standard Train Test Split**

In [69]:

# def split_data(df, train_size_=0.70, valid_size_=0.15, test_size_=0.15, random_state=42):
#     # Log-transform y
#     df=df.copy()
#     y = df.pop("NILVAL_LONG_USD")
#     y = np.log1p(y)
#     X = df
#     # scaler = StandardScaler()
#     # X_scaled = scaler.fit_transform(X) 

#     # Split the data
#     X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=valid_size_ + test_size_, 
#     shuffle = True, random_state=random_state)
#     X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=test_size_ / (valid_size_ + test_size_), shuffle = True, random_state=random_state)

#     # X_train = pd.DataFrame(X_train, columns=X.columns)
#     # X_valid = pd.DataFrame(X_valid, columns=X.columns)
#     # X_test = pd.DataFrame(X_test, columns=X.columns)

#     return X_train, y_train, X_valid, y_valid, X_test, y_test

In [70]:
# X_train_bb, y_train_bb, X_valid_bb, y_valid_bb, X_test_bb, y_test_bb = split_data(df_bb)
# X_train_fb, y_train_fb, X_valid_fb, y_valid_fb, X_test_fb, y_test_fb = split_data(df_fb)

# print((X_train_bb.shape,y_train_bb.shape ),(X_valid_bb.shape, y_valid_bb.shape),(X_test_bb.shape, y_test_bb.shape))
# print((X_train_fb.shape,y_train_fb.shape ),(X_valid_fb.shape, y_valid_fb.shape),(X_test_fb.shape, y_test_fb.shape))

In [71]:

# # Extract target and features
# Target = 'NILVAL_LONG_USD'
# Predictors = ['GRADE', 'AGE', 'SKILL', 'NUMOFF', 'POS', 'HEIGHT_IN', 'WEIGHT_LBS',
#        'COLLDIST_MI', 'INSTA_LONG', 'TWIT_LONG', 'TIK_LONG',
#        'TOT_FOL', 'RECRUIT_YEAR', 'EXP_MONTHS', 'EXP_YEARS',
#        'ClassificationCode', 'REV_MEN', 'EXP_MEN']



**XGBoostRegressor Base Model Decision Trees as Base Learners- using all features- using scikit learn**

In [72]:
#using default parameters, boosting rounds = 5  objective =reg:squarederror  XGBoost default is gbtree
xg_reg2 = XGBRegressor(n_estimators=5, random_state= 42)
xg_reg2.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=5, n_jobs=0,
             num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=42, reg_alpha=0, ...)

In [73]:
# Predict the labels of the test set: preds
preds = xg_reg2.predict(X_test)

# compute the rmse: rmse
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

 
#Measuring Goodness of fit in Training data
from sklearn import metrics
print('Train R2 Value:',metrics.r2_score(y_train, xg_reg2.predict(X_train)))
 
#Measuring accuracy on Testing Data
print('Accuracy Test',100- (np.mean(np.abs((y_test - preds) / y_test)) * 100))



#Plotting the feature importance for Top 10 most important columns
%matplotlib inline
feature_importances = pd.Series(xg_reg2.feature_importances_, index=Predictors)
feature_importances.nlargest(10).plot(kind='barh')
plt.xlabel ("Feature Importance")
plt.title("Top 10 Most Important Features - XGBoost Regressor Model (GBtree)")
 
#Printing some sample values of prediction
TestingDataResults=pd.DataFrame(data=X_test, columns=Predictors)
TestingDataResults[Target]=y_test
TestingDataResults[('Predicted'+Target)]=preds
TestingDataResults.head()

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.scatter(y_test, preds)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linewidth=2)
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title('True vs Predicted Values')
plt.show()


RMSE: 232261.869339
Train R2 Value: 0.7381574457827316
Accuracy Test 22.42118239814566


ValueError: Length of values (17) does not match length of index (18)

In [ ]:
# metrics.classification_report(y_test, pred)

In [ ]:
# # https://github.com/parrt/dtreeviz/blob/master/notebooks/dtreeviz_xgboost_visualisations.ipynb

# #initialize dtreevizmodel (adaptor)

# viz_rmodel = dtreeviz.model(model=xg_reg2, tree_index=1, 
#                             X_train=df_football[Predictors],
#                             y_train=df_football[Target], 
#                             feature_names=Predictors, 
#                             target_name=Target)

In [ ]:
# #Tree structure visualizations

# # viz_rmodel.view()
# viz_rmodel.view(orientation="LR")

In [ ]:
# #Prediction path explanations
# x = df_football[Predictors].iloc[10]
# x
# viz_rmodel.view(x=x)

In [ ]:
# viz_rmodel.view(show_just_path=True, x = x)

In [ ]:
# print(viz_rmodel.explain_prediction_path(x))

In [ ]:
# viz_rmodel.plot_importance(x, figsize=(3.5,2))

In [ ]:
# #leaf ratios
# viz_rmodel.leaf_sizes()

In [ ]:
# viz_rmodel.rtree_leaf_distributions()

In [ ]:
# viz_rmodel.node_stats(node_id=4)

**XGBoost with linear learner base model (GBlinear)**- Have to use XGBoot non scikit learn functions to build model

In [ ]:
DM_train = xgb.DMatrix(data=X_train, label=y_train, enable_categorical= True)
DM_test = xgb.DMatrix(data=X_test, label=y_test, enable_categorical = True)


# Create the parameter dictionary: params
params = {"booster":"gblinear", "objective":"reg:squarederror"}

# Define evaluation data
eval_data = [(DM_train, 'train'), (DM_test, 'test')]


# Train the model: xg_gbl
xg_gbl = xgb.train(params=params, dtrain=DM_train, evals =eval_data, num_boost_round=5)


# Predict the labels of the test set: preds
preds = xg_gbl.predict(DM_test)

# Calculate the error RMSE
error = xg_gbl.eval(DM_test)
print('Evaluation Error:', error)

# Compute and print the RMSE
rmse = np.sqrt(mean_squared_error(y_test, preds))
# compute the rmse: rmse
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE %f:" % (rmse))
#Measuring Goodness of fit in Training data
from sklearn import metrics
print('Train R2 Value:',metrics.r2_score(y_train, xg_gbl.predict(DM_train)))
#Measuring accuracy on Testing Data
print('Accuracy Test',100- (np.mean(np.abs((y_test - preds) / y_test)) * 100))


# # Plot the feature importance
from matplotlib import pyplot
# plt.figure(figsize=(10, 20))
plot_importance(xg_gbl, max_num_features = 15)
pyplot.show()



import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.scatter(y_test, preds)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linewidth=2)
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title('True vs Predicted Values')
plt.show()



In [ ]:
y_test_trans = y_test.reshape(-1)
residuals = y_test_trans - preds
residuals
# preds.shape
# # y_test.shape

plt.figure(figsize=(10, 5))
plt.hist(residuals, bins= 20)
plt.xlabel('Residual')
plt.ylabel('Frequency')
plt.title('Distribution of Residuals')
plt.show()



In [ ]:
nil_dmatrix = xgb.DMatrix(data=X, label=y)

# Perform cross-valdiation: cv_results
cv_results = xgb.cv(dtrain=nil_dmatrix, params=params, nfold=4,
                    num_boost_round=100, metrics='rmse', as_pandas=True, seed=42)

# Print cv_results
print(cv_results)

# Extract and print final boosting round metric
print((cv_results['test-rmse-mean']).tail(1))



In [ ]:
nil_dmatrix = xgb.DMatrix(data=X, label=y)

# Perform cross-valdiation: cv_results
cv_results = xgb.cv(dtrain=nil_dmatrix, params=params, nfold=4,
                    num_boost_round=100, metrics='mae', as_pandas=True, seed=42)

# Print cv_results
print(cv_results)

# Extract and print final boosting round metric
print((cv_results['test-mae-mean']).tail(1))



In [ ]:
# https://github.com/parrt/dtreeviz/blob/master/notebooks/dtreeviz_xgboost_visualisations.ipynb

#initialize dtreevizmodel (adaptor)

# viz_rmodel = dtreeviz.model(model=xg_reg2, tree_index=0, 
#                             X_train=X, 
#                             y_train=y, 
#                             feature_names=Predictors, 
#                             target_name=Target)

**Finetune XGBoost Model- scikit learn**<br> 9 hyperparameters to address bias-variance trade-off:
<br>Iteration Control: num_boost_round; early_stopping_round
<br>Params:
<br> Shrinkage: eta
<br> Tree-Booster constraints: max_depth; min_child_weight; gamma
<br> Random Subsampling: subsample; colsample_bytree
<br> L1 & L2 regularizations: alpha; lambda
<br> Fixed paramaters: objective function for regression ;eval_metric; tree_method (hist; )


**Pre-tuning regression to compare initial performance curves of datasets along training iterations**
<br> Compare training and test performance to assess bias-variance tradeoff(problem of under/over fitting) at initial state
<br> Use for pre/post tuning performance curves comparison to assess regularization impact of tuning. Is tuning effectively performing regularization on model to assess bias-variance tradeoff?

In [ ]:
Target = 'NILVAL_LONG_USD'
feature_name = ['GRADE', 'AGE', 'SKILL', 'NUMOFF', 'POS', 'HEIGHT_IN', 'WEIGHT_LBS',
       'COLLDIST_MI', 'INSTA_LONG', 'TWIT_LONG', 'TIK_LONG',
       'TOT_FOL', 'RECRUIT_YEAR', 'EXP_MONTHS', 'EXP_YEARS',
       'ClassificationCode', 'REV_MEN', 'EXP_MEN']





In [ ]:
#params dictionary-baseline parameter to tune
# Define hyperparameters
eval_metrics = []
params = {
    'max_depth':6,
    'min_child_weight': 1,
    'eta': 0.3,
    'subsample': 1,
    'colsample_bytree': 1,
    'gamma': 0, 
    'reg_alpha': 0.1,
    'reg_lambda': 1,
     'eval_metric': ['rmse', 'mae'],
    # Other parameters-fixed
    'objective': 'reg:squarederror',
    'tree_method': 'hist',
    'seed' : 42
}
num_boost_round = 1000
early_stopping_rounds = 20   #10

# Define evaluation metrics (metrics)
metrics = {'rmse', 'mae'}

seed = 42

# DM_train = xgb.DMatrix(data=X_train, label=y_train, enable_categorical= True)
# DM_test = xgb.DMatrix(data=X_test, label=y_test, enable_categorical = True)


def bias_variance_performance(params, X_train, y_train, X_test, y_test):
    
    DM_train = xgb.DMatrix(data=X_train, label=y_train, enable_categorical= True)
    DM_test = xgb.DMatrix(data=X_test, label=y_test, enable_categorical = True) 
   
    evals_result = {}
    model = xgb.train(
        params,
        dtrain=DM_train,
        num_boost_round=num_boost_round,
        evals=[(DM_train, "Train"), (DM_test, "Test")],
        early_stopping_rounds=early_stopping_rounds,
        evals_result=evals_result,
    
    )
    train_error = evals_result["Train"]["mae"]
    test_error = evals_result["Test"]["mae"]
    train_rmse = evals_result["Train"]["rmse"]
    test_rmse = evals_result["Test"]["rmse"]
    
  
    best_mae = min(evals_result['Test']['mae'])
    best_mae_round = evals_result['Test']['mae'].index(best_mae) + 1  # Add 1 to get the round number
    
    best_rmse = min(evals_result['Test']['rmse'])
    best_rmse_round = evals_result['Test']['rmse'].index(best_rmse) + 1  # Add 1 to get the round number
    
    print("Best MAE before the Tuning: {:.4f} with {} rounds".format(best_mae, best_mae_round))
    print("Best RMSE before the Tuning: {:.4f} with {} rounds".format(best_rmse, best_rmse_round))
    
    return model, evals_result

# num_boost_round = 1000
# early_stopping_rounds = 60
pretuned_model, pre_evals_result = bias_variance_performance(params, X_train, y_train, X_test, y_test)

Gap for MAE for pre-tuned hyperparametes suggests overfitting
<br>**Plot performance curve for pre-tune MAE and RMSE**

In [ ]:
#Extract MAE values for both training and test sets
train_mae = pre_evals_result['Train']['mae']
test_mae = pre_evals_result['Test']['mae']


#Extract RMSE values for both training and test sets
train_rmse = pre_evals_result['Train']['rmse']
test_rmse = pre_evals_result['Test']['rmse']

# Create a plot to visualize the MAE performance curve
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_mae) + 1), train_mae, label='Train MAE')
plt.plot(range(1, len(test_mae) + 1), test_mae, label='Test MAE')
plt.xlabel('Boosting Round')
plt.ylabel('Mean Absolute Error (MAE)')
plt.title('Pre-Tuning XGBoost - MAE Performance Curve')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_rmse) + 1), train_rmse, label='Train RMSE')
plt.plot(range(1, len(test_rmse) + 1), test_rmse, label='Test RMSe')
plt.xlabel('Boosting Round')
plt.ylabel('Root Squared Mean Error (RMSE)')
plt.title('Pre-Tuning XGBoost - RMSE Performance Curve')
plt.legend()
plt.grid(True)

As the iterations went on, the initial (pre-tuned) training performance continued improving, while the initial (pre-tuned) test performance stagnated at some point and then started increasing. 
<b> This suggests a salient variance, or over-fitting, in the initial (pre-tuned) model

What is feature importance of model?

In [ ]:


# Obtain the feature importance scores
importance_type = "weight"  # You can use "weight," "gain," or "cover"
importances = pretuned_model.get_score(importance_type=importance_type)

# Sort the features by importance in descending order
sorted_feature_names = sorted(importances, key=importances.get, reverse=True)

# Map sorted_feature_names to the corresponding feature names
mapped_feature_names = [feature_name[sorted_feature_names.index(name)] for name in sorted_feature_names]



mapped_feature_names
# # # Create a feature importance plot with actual feature names as y-axis labels
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(9, 6))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_importance(pretuned_model, ax=ax, height=0.6, importance_type=importance_type)
    ax.set_yticklabels(mapped_feature_names)
    ax.set_title('XGBoost Regression -Pre-tuning Feature Importance')


    


**K-Cross Fold Validation**
<br> Use built in xgb.cv() function- performed only on TRAIN set

In [ ]:

def perform_k_fold_cross_validation(params, DM_train, metrics, num_boost_round, early_stopping_rounds, num_folds=5):
    """
    Perform k-fold cross-validation for an XGBoost model.

    Parameters:
    - params: XGBoost hyperparameters
    - DM_train: DMatrix for the training data
    - metrics: List of evaluation metrics
    - num_boost_round: Number of boosting rounds
    - early_stopping_rounds: Early stopping criteria
    - num_folds: Number of cross-validation folds (default is 5)

    Returns:
    - cv_results: Cross-validation results as a pandas DataFrame
    """
    cv_results = xgb.cv(
        params=params,
        dtrain=DM_train,
        num_boost_round=num_boost_round,
        seed=42,  # Seed for randomization
        nfold=num_folds,
        metrics=metrics,
        early_stopping_rounds=early_stopping_rounds
    )
    return cv_results

# Call the cross_validation function
cv_results = perform_k_fold_cross_validation(params, DM_train, metrics, num_boost_round, early_stopping_rounds)


In [ ]:
cv_results
best_round = cv_results['test-rmse-mean'].idxmin()
best_rmse = cv_results['test-rmse-mean'].min()
best_mae = cv_results['test-mae-mean'][best_round]

print("Best RMSE: {:.4f} at boosting round {}.".format(best_rmse, best_round))
print("Best MAE: {:.4f} at boosting round {}.".format(best_mae, best_round))

In [ ]:

num_boost_rounds = len(cv_results)


plt.figure(figsize=(10, 6))
plt.plot(range(1, num_boost_rounds + 1), cv_results['train-mae-mean'], label='Train MAEE', )
plt.plot(range(1, num_boost_rounds + 1), cv_results['test-mae-mean'], label='Test MAE', )
plt.xlabel('Boosting Rounds')
plt.ylabel('MAE')
plt.title('K Fold Learning Curve - MAE')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(range(1, num_boost_rounds + 1), cv_results['train-rmse-mean'], label='Train RMSE', )
plt.plot(range(1, num_boost_rounds + 1), cv_results['test-rmse-mean'], label='Test RMSE', )
plt.xlabel('Boosting Rounds')
plt.ylabel('RMSE')
plt.title('K Fold Learning Curve - RMSE')
plt.legend()
plt.grid(True)
plt.show()

**Use GridSearchCV to find best parameters**

**GridSearch for all params above takes a long time- try pairwise grid search**

In [ ]:
best_score = float('inf')  # Initialize with a high value
best_params = params.copy()  # Initialize with the baseline parameters

param_grid = {
    'max_depth': [3, 4, 5, 6, 7],
    'min_child_weight': [1, 2, 3, 4, 5],
    'eta': [0.01, 0.1, 0.2, 0.3, 0.4],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_alpha': [0, 0.1, 0.2, 0.3],
    'reg_lambda': [0.1, 1, 10, 100],
    'eval_metric': ['rmse', 'mae'],
    # Other parameters-fixed
    'objective': ['reg:squarederror'],
    'tree_method': ['hist']
}

# Define sets of hyperparameters to tune as pairs
parameter_sets = [
    ('max_depth', 'eta'),
    ('min_child_weight', 'gamma'),
    ('colsample_bytree', 'subsample'),
    ('reg_alpha', 'reg_lambda')
]

for param_set in parameter_sets:
    param1, param2 = param_set

    for val1 in param_grid[param1]:
        for val2 in param_grid[param2]:
            # Create a copy of the current parameter settings
            params_copy = best_params.copy()

            # Set the values for the current pair of hyperparameters
            params_copy[param1] = val1
            params_copy[param2] = val2

            # Perform grid search using the current hyperparameters
            model, evals_result = bias_variance_performance(params_copy)
            mse = min(evals_result['Test']['mae'])  # Use an appropriate metric

            # Check if this combination of hyperparameters is better
            if mse < best_score:
                best_score = mse
                best_params[param1] = val1
                best_params[param2] = val2

print("Best Hyperparameters:", best_params)
print("Best Score (MAE):", best_score)

In [ ]:

print("Parameters after the Pair Wise Tuning")
best_params

In [ ]:
#Now run tuned hyperparmeters (best_params) 
# num_boost_rounds = 1000
# early_stopping_rounds = 94
pw_model_1, pw_evals_result_1 = bias_variance_performance(best_params)

In [ ]:
#Extract MAE values for both training and test sets
train_mae = pw_evals_result_1['Train']['mae']
test_mae = pw_evals_result_1['Test']['mae']


#Extract RMSE values for both training and test sets
train_rmse = pw_evals_result_1['Train']['rmse']
test_rmse = pw_evals_result_1['Test']['rmse']

# Create a plot to visualize the MAE performance curve
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_mae) + 1), train_mae, label='Train MAE' )
plt.plot(range(1, len(test_mae) + 1), test_mae, label='Test MAE' )
plt.xlabel('Boosting Round')
plt.ylabel('Mean Absolute Error (MAE)')
plt.title('Post-Tuning XGBoost - MAE Performance Curve')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_rmse) + 1), train_rmse, label='Train RMSE' )
plt.plot(range(1, len(test_rmse) + 1), test_rmse, label='Test RMSE')
plt.xlabel('Boosting Round')
plt.ylabel('Root Squared Mean Error (RMSE)')
plt.title('Post-Tuning XGBoost - RMSE Performance Curve')
plt.legend()
plt.grid(True)

In [ ]:


# Obtain the feature importance scores
importance_type = "weight"  # You can use "weight," "gain," or "cover"
importances = pw_model_1.get_score(importance_type=importance_type)

# Sort the features by importance in descending order
sorted_feature_names = sorted(importances, key=importances.get, reverse=True)

# Map sorted_feature_names to the corresponding feature names
mapped_feature_names = [feature_name[sorted_feature_names.index(name)] for name in sorted_feature_names]



mapped_feature_names
# # # Create a feature importance plot with actual feature names as y-axis labels
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(9, 6))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_importance(pw_model_1, ax=ax, height=0.6, importance_type=importance_type)
    ax.set_yticklabels(mapped_feature_names)
    ax.set_title('XGBoost Regression -Post-tuning Feature Importance')

#go back and change when early stopping should happend  60???

**Evaluation** During boosting rounds, the model object has learned all the patterns of the training set. Perform testing.

In [ ]:
preds1 = pretuned_model.predict(DM_test)
preds2 = pw_model_1.predict (DM_test)


rmse = mean_squared_error(y_test, preds1, squared=False)

rmse2 = mean_squared_error(y_test, preds2, squared=False)

print(f"RMSE of the baseline model: {rmse:.3f}")
print(f"RMSE of the tuned model: {rmse2:.3f}")

mae = mean_absolute_error(y_test, preds1)
mae2 = mean_absolute_error(y_test, preds2)
print(f"MAE of the baseline model: {mae:.3f}")
print(f"MAE of the tuned model: {mae2:.3f}")

# # Calculate the mean of the target variable in the test data
# y_mean = np.mean(y_test)

# # Calculate the total sum of squares (TSS)
# TSS = np.sum((y_test - y_mean)**2)

# # Calculate the residual sum of squares (RSS)
# RSS = np.sum((y_test - preds1)**2)
# RSS2 = np.sum((y_test - preds2)**2)

# # Calculate R-squared (R^2)
# r_squared = 1 - (RSS / TSS)
# r_squared2 = 1 - (RSS2 / TSS)

# print("Baseline R-squared (R^2) =", r_squared)
# print("Tuned R-squared (R^2) =", r_squared2)




In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(y_test, preds1, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(preds1), max(y_test))
p2 = min(min(preds1), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(y_test, preds1)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linewidth=2)
plt.xlabel('True Values')
plt.ylabel('Predicted NIL Values')
plt.title('True vs Predicted NIL Values- Baseline XGBRegressor')
plt.show()

In [ ]:
# residuals = y_test - preds1

# plt.figure(figsize=(10, 5))
# plt.hist(residuals, bins=20)
# plt.xlabel('Residual')
# plt.ylabel('Frequency')
# plt.title('Distribution of Residuals')
# plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(y_test, preds2, c='crimson')
plt.yscale('log')
plt.xscale('log')

p11 = max(max(preds2), max(y_test))
p22 = min(min(preds2), min(y_test))
plt.plot([p11, p22], [p11, p22], 'b-')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(y_test, preds2)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linewidth=2)
plt.xlabel('True NIL Values')
plt.ylabel('Predicted NIL Values')
plt.title('True vs Predicted NIL Values- Tune XGBRegressor')
plt.show()

In [ ]:
# residuals = y_test - preds2

# plt.figure(figsize=(10, 5))
# plt.hist(residuals, bins=20)
# plt.xlabel('Residual')
# plt.ylabel('Frequency')
# plt.title('Distribution of Residuals')
# plt.show()